#Second Practical work

This is second practical work skeleton.

It is divided in 3 parts_

1.   Python module load section: load all the needed modules
2.   Google Drive access and mounting
3.   Host load data
4.   GPU load





##Module load section
We will install the pycuda module in the colaboratory environment, to combine python and CUDA in our code. Because pycuda is not available by default, this task can take a few minutes.

As well we load numpy and matplotlib modules.

In [ ]:
!pip install pycuda

We import the Python modules which we will use in the notebook (numpy, pycuda and time module) 

In [ ]:
import  numpy  as  np
import  pycuda.autoinit
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv
import  pycuda.gpuarray as  gpuarray
from sys import getsizeof
import time

In [ ]:
%pylab inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile

##Mount Google Drive and access grant
We use Google API to allow access to our Google Drive.

For security and privacy reasons, this permission is temporally, and will be requested every time you restart the notebook. 

Your Google Drive will be mounted in the "/content/drive" folder. The first Folder will be "My Drive".

You have to copy your images to a folder in the "My Drive" folder in your Google Drive.

To check your available files, use the cell indicated as "**List Files in Folder**"

In [ ]:
from os.path import join
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
MYDRIVE="/content/drive/My Drive/"

**List Files in Folders**

In [ ]:
!ls /content/drive/My\ Drive/images

We define a variable IMAGE which is constructed joining the "MYDRIVE" varible plus the final path to your image.

In [ ]:
IMAGE=join(MYDRIVE,HERE YOU HAVE TO WRITE THE IMAGE TO LOAD)

In [ ]:
print("Image to load: {0}".format(IMAGE))

Loads an image in a numpy variable, and show.

In [ ]:
image=np.array(Image.open(IMAGE))
plt.figure()
plt.imshow(image)

#FILTER KERNEL

Here you have to define your image_filter GPU kernel.

In [ ]:
kernel  =  SourceModule ("""
__global__ void image_filter( float * image,          //Source GPU array floating point 32 bits,
		                          float * filter_mask,    //Filter Mask GPU array 2D floating point 32 bits
                              float * filtered_image,  //Target GPU array 2D floating point 32 bits,
                              int NumRowsImg,         //Image Numrows,
                              int NumColsImg,         //Int32 Image Numcolumns,
                              int NumRowsFilter,       //Int32 Image NumRows filter mask,
                              int NumColsFilter       //Int32 Image NumCols filter mask
                              ) 
{ 
    //This instruction calculate in the offset in the memory data, based on the block identificator blockIdx.x
    // YOU HAVE TO WRITE HERE YOUR FILTER KERNEL USING TILED MEMORY ALGORITHMS

    //THIS CODE JUST COPY THE SOURCE IMAGE IN THE TARGET VECTOR
    int idxY;
    int idxX;
    int vX;

    idxY = blockIdx.y*blockDim.y+threadIdx.y; //With this we calculate the row address in target matrix
    idxX = blockIdx.x*blockDim.x+threadIdx.x;
    if ((idxY<NumRowsImg) && (idxX<NumColsImg)){
      vX=idxY*NumColsImg+idxX;
      filtered_image[vX]=image[vX];
    }
}

""")

And import the kernel definition to python

In [ ]:
image_filter = kernel.get_function ('image_filter')

Recover the image shape, and select the two first items (rows and columns).

In [ ]:
image_shape=image.shape[0:2]

Set block size and grid size.

In [ ]:
block_s=16
block_size=(block_s,block_s,1)
grid_size=(int(np.ceil(image_shape[1]/block_s)),int(np.ceil(image_shape[0]/block_s)))

Allocates the target image memory.

In [ ]:
result_image=np.zeros(image.shape,dtype=np.uint8)

In [ ]:
depth = image.shape[2]

Here we defines our sample filter masks.

In [ ]:
filter1=np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])
filter2=np.array([[0.5, 0 , -0.5]])
filter3=np.array([[0.5],[0],[-0.5]])

filter4=np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]
])
filter5=np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]
])

We select the filter mask, and upload to the GPU device.

In [ ]:
filter_mask=filter1.astype(np.float32)
filter_mask_gpu=gpuarray.to_gpu(filter_mask)

##Prepares the execution environment 

Allocate the temporal filtered memory layer, where will be store the filtered image.

In [ ]:
filtered_image_layer_gpu = gpuarray.zeros(image_shape,np.float32)

Here defines the main block.
Walk through the layers. Copying the selected layer to temporal image_layer, upload the layer to the GPU and invokes the image_filter kernel.

In [ ]:
start_t = time.time()
for layer in range(depth):
  image_layer = np.zeros(image_shape,dtype=np.float32) #Allocate the temporal image layer
  image_layer=(image[:,:,layer]).astype(np.float32)    #Copy from the orignal image the selected layer into the temporal memory
  image_layer_gpu=gpuarray.to_gpu(image_layer)         #Uploads the image layer to the GPU
  filtered_image_layer_gpu.fill(0.0)                   #Fills with 0 the target memory in the GPU
  image_filter(                                        #invokes the kernel
          image_layer_gpu,
          filter_mask_gpu,
          filtered_image_layer_gpu,
          np.int32(image_shape[0]),
          np.int32(image_shape[1]),
          np.int32(filter_mask.shape[0]),
          np.int32(filter_mask.shape[1]),
          block=block_size,
          grid=grid_size
        )
  filtered_image_layer=filtered_image_layer_gpu.get() #Download from the GPU the filtered image
  result_image[:,:,layer] = filtered_image_layer.astype(np.uint8) #Copy the filtered image to the final image store, 
                                                                  #casting the data type from double to uint8
end_t = time.time()

#Show results


In [ ]:
print("Execution time is: {0} seconds.".format(end_t-start_t))

In [ ]:
plt.figure()
plt.imshow(result_image)